In [1]:
import torch
from torch import nn
import numpy as np

In [2]:
text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

In [4]:
int2char

{0: 'u',
 1: 'n',
 2: 'v',
 3: 'c',
 4: 'o',
 5: 'i',
 6: 'a',
 7: 'h',
 8: 'e',
 9: 'f',
 10: 'w',
 11: 'r',
 12: 'd',
 13: 'm',
 14: ' ',
 15: 'g',
 16: 'y'}

In [5]:
char2int

{'u': 0,
 'n': 1,
 'v': 2,
 'c': 3,
 'o': 4,
 'i': 5,
 'a': 6,
 'h': 7,
 'e': 8,
 'f': 9,
 'w': 10,
 'r': 11,
 'd': 12,
 'm': 13,
 ' ': 14,
 'g': 15,
 'y': 16}

In [6]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 15 characters


In [7]:
# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

In [8]:
text

['hey how are you', 'good i am fine ', 'have a nice day']

In [10]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


In [11]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

In [15]:
input_seq

[[7, 8, 16, 14, 7, 4, 10, 14, 6, 11, 8, 14, 16, 4],
 [15, 4, 4, 12, 14, 5, 14, 6, 13, 14, 9, 5, 1, 8],
 [7, 6, 2, 8, 14, 6, 14, 1, 5, 3, 8, 14, 12, 6]]

In [13]:
target_seq

[[8, 16, 14, 7, 4, 10, 14, 6, 11, 8, 14, 16, 4, 0],
 [4, 4, 12, 14, 5, 14, 6, 13, 14, 9, 5, 1, 8, 14],
 [6, 2, 8, 14, 6, 14, 1, 5, 3, 8, 14, 12, 6, 16]]

In [14]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [16]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

Input shape: (3, 14, 17) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


In [20]:
target_seq

[[8, 16, 14, 7, 4, 10, 14, 6, 11, 8, 14, 16, 4, 0],
 [4, 4, 12, 14, 5, 14, 6, 13, 14, 9, 5, 1, 8, 14],
 [6, 2, 8, 14, 6, 14, 1, 5, 3, 8, 14, 12, 6, 16]]

In [21]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [23]:
target_seq

tensor([[ 8., 16., 14.,  7.,  4., 10., 14.,  6., 11.,  8., 14., 16.,  4.,  0.],
        [ 4.,  4., 12., 14.,  5., 14.,  6., 13., 14.,  9.,  5.,  1.,  8., 14.],
        [ 6.,  2.,  8., 14.,  6., 14.,  1.,  5.,  3.,  8., 14., 12.,  6., 16.]])

In [24]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden